<a href="https://colab.research.google.com/github/sarrahrose04/PovertyMapping/blob/main/Processing%26Downloading_DaytimeImg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount{'/content/gdrive', force_remount=True}

In [ ]:
#Ensure that all edits in libraries are autmatically reloaded & all images shown in notebook

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install earthengine-api
!earthengine authenticate 
import ee; ee.Initialize();


**Dataset Preparation**

In [ ]:
#Import CSV 
import pandas as pd
centroid_csv_path = '' #paste

#Store in pandas dataframe
df = pd.read_csv(centroid_csv_path)

In [ ]:
#Set the id as row index df
df = df.set_index('id')
df.head()

#Determine row count & save as imagery count
imagery_count = df.count()[1] + 1
df.count()

In [ ]:
! pip install 'geopandas'

**Loading Shapefile**

In [ ]:
import geopandas as gpd 

adm0_shp = gpd.read_file('') #paste
#Only one row because it's a country-level shapefile
adm0_shp.head() 

**Generate Bounding Box Polygon**


In [ ]:
#Create a bounding box polygon using geopanda's function envelope 
bbox_poly = adm_shp.geometry.envelope 
bbox = bbox_poly.to_json() #Convert to JSON 

#Extract BB Coordinates from JSON Object 
bbox_dict = eval(bbox) #convert JSON obj to Dict Obj
bbox_features_dict = bbox['features'][0] #subset of first feature containing coordinates
bbox_coordinates = bbox_features_dict['geometry']['coordinates'] #subset of dictionary to get coordinate values of BB

#Convert Bounding Box into GEE polygon object
bounding_box = ee.Geometry.Polygon(bbox_coordinates)

**Satellite Imagery Filtering & Visualisation**

Input Data Information

In [ ]:
country = 'THA'
year = '2015'

In [ ]:
#Use If-else statements to decide parameters of satellite imagery 
if int(year) >= 2015: 
  day_sat='ST' #sentinel 2 satellite
  img_res = '384'
  img_size = str(int(img_res)*10)
else: 
  day_sat = "LS" #Landsat
  img_res = 256
  img_size = str(int(img_res)*15)

#Generate Output Directory
drive_folder = "_".join(["CNN","IMGB", country, year, day_sat, img_res,"TIF", img_size])

#Assemble DIMG filename
DIMG = "_".join(["CNN_DIMG", country, year, day_sat, img_res, img_size])

#Print values to check
print(day_sat)
print(img_res)
print(img_size)
print(drive_folder)
print(DIMG)

In [2]:
#Specify coverage date

#Start
start_MM = "01"
start_DD = "01"
start_date = "-".join([year,start_MM,start_DD])
print("Coverage start date: "+start_date)

#End (change if comp. img for entire country is incomplete)
end_YYYY = "2016"
end_MM = "04"
end_DD = "30"

if int(end_YYYY)<int(year): 
  end_YYYY=str(int(year)+1)
  print("Please specify end_YYYY")

end_date = "-".join([end_YYYY, end_MM, end_DD])
print("Coverage end date: " +end_date)



**Cloud Masking**

Most remote sensing datasets come with a QA or Cloud Mask band that contains the information on whether pixels is cloudy or not. 

Masking pixels in an image makes those pixels transparent and excludes them from analysis. 

https://developers.google.com/earth-engine/tutorials/tutorial_api_05

In [ ]:
import folium 

#Select appropriate filter for satellite to be used 
#Satellite selected based on availability of image coverage (LS7 1999 - present; LS8 2013 - pres; ST2 2015 - pres)
if day_sat == "ST":

  def maskS2clouds(image): 
    qa = image.select('QA60')
  #Bits 10 and 11 are clouds and cirrus, respecitvely
    cloudBitMask = 1 << 10
    cirrusbitMask = 1 <<11
  #Both flags should be set to zero, indicating clear conditions 
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

  #Defines Visualisation Parameters
  rgbVis = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}

  #Filter an image collection
  cloud_masked = ee.ImageCollection('COPERNICUS/S2').filterDate(start_date, end_date)\
  .filterBounds(bounding_box).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60))\
  .map(maskS2clouds)

  #Take median value
  satellite_imagery = cloud_masked.median().visualize(**rgbVis) 

filterDate() defines the temporal coverage

filterBounds() uses the bounding box to limit the filter to the country boundaries

filter(ee.Filter.lt(‘CLOUDY_PIXEL_PERCENTAGE’, 60)) provides the filter to exclude
images with more than 60% cloud cover

map(maskS2clouds) uses the function for creating cloud mask

In [ ]:
else: 
  if int(year) < 2013: 
    landsat_mission = 'LANDSAT/LE07/C01/T1' #Imagery collection name in GEE
    LS_day_sat = 'LS7'
  else: 
    landsat_mission = 'LANDSAT/LC08/C01/T1'
    LS_day_sat = 'LS8'
  
  filtered_shp = ee.ImageCollection(landsat_mission)\
  .filterDate(start_date, end_date)\
  .filterBounds(bounding_box)

  #Use inbuilt EE function to create big composite image from Landsat tiles
  composite = ee.Algorithms.Landsat.simpleComposite(filtered_shp).float();

  #Pansharpening

  #Select RGB bands
  if LS day_sat == 'LS7': 
    rgb = composite.select('B3', 'B2', 'B1').unitScale(0,255)
  if LS day_sat == 'LS8': 
    rgb = composite.select('B4', 'B3', 'B2').unitScale(0,255)
  
  #Select panchromatic band
  gray = composit.select('B8').unitScale(0,155)

  #Convert RGB image to Hue Saturation Value & select only hue & saturation bands
  huesat = rgb.rgbToHsv().select('hue','saturation')
  #Combine hue, saturation & panchromatic bands, then convert back to RGB for upscaled image
  satellite_imagery = ee.Image.cat(huesat, gray).hsvToRgb()

Pansharpening combines high resolution panchromatic images (black and
white but sensitive to colors) with lower resolution multispectral band images.

Creating a Folium Map

In [ ]:
#get centroid coordinates of bounding box for map view centering
cen_x = bbox_poly.centroid.x[0]
cen_y = bbox.poly.centroid.y[0]

#create folium object
map = folium.Map(location = [cen_y], cen_x], 
                 zoom_start = 6, #defines zoom level of map
                 width = 1280, #define size of map
                 height = 766,
                 attr=day_sat) #display name of satellite

#get mapID of sat_imagery from GEE
ee_image_map_id = ee.Image(satellite_imagery).getMapId()

#add sat_imagery to map 
folium.raster_layers.TileLayer(
    tiles = ee_image_map_id['tile_fetcher'].url_format, #map data source; uses mapID to get URL link of filtered satellite imagery
    attr = 'Google Earth Engine', 
    name = 'Daytime Imagery', 
    overlay = True, #imagery will be placed over Folium default base map
    control = True, #layer will be included in Layer Control 
    ).add_to(map)

#Add bounding box
folium.GeoJson(
    data = bounding_box.getInfo(),
    name = 'Bounding box',
    style_function = lambda feature: {
        'fillColor': '#FFFFFF00',
        'weight' : 3,
        'fillOpacity' : 0.5,
        },
    overlay = True, 
    control = True,   
    ).add_to(map)

#Add map title 
if day_sat == "ST": 
  map_title = "Sentinel-2 Imagery: Please check if composite image is complete"
else: 
  map_title = "Landsat Imagery: Please check if composite image is complete"
title_html = '''
              <h3 align = "center" style="font-size:16px"><b>{}</b></h3>'''.format(map_title)
map.get_root().html.add_child(folium.Element(title_html))

#add layer control panel 
map.add_child(folium.LayerControl())
#Display map
display(map)

Checking Task Count

In [ ]:
#Identify the number of “Ready” and “Running” tasks from the GEE task list
#Verify less than 3000

def get_queued_task():
  queued_task_count = 0
  for queued_task in ee.batch.Task.list(): 
    if queued_task.state in ["READY", "RUNNING"]: 
      queued_task_count += 1
  return queued_task_count

def get_queued_task_filenames():
  print("Fetching queued files")
  task_filenames = []
  for queued_task in ee.batch.Task.list():
    if queued_task.state in ["READY", "RUNNING"]: 
      print(queued_task.state+": "+queued_task.status()['description'])
      task_filenames.append(queued_task.status()['description'])
  print("---end fetch---\n")


Downloading Imagery

In [ ]:
import os
def download_satellite_imagery(sat_imagery): #satellite imagery object
  next_batch_size = 10 #set no. of new tasks to be added after reaching task limit
  target count = 3000 - next_batch_size #Threshold before creating new tasks

  task_count = get_queued_task() #Execute function & store R/R tasks in var.
  queued_filenames = get_queued_task_filenames() #Execute function & store
  print('Number of active tasks: {: }.'.format(task_count)) #Print active tasks

for i in range(1,imagery_count):

  imagery_file = DIMG + '_{:06d}.format(i)
  imagery_filepath = '/content/gdrive/MyDrive/' + drive_folder + '/' + imagery_file + '.tif'

  if task_count == 3000: #Number of tasks has reached limit

  #Loop until task count has not reach target_count
    while task_count > target count: 
      active_task = get_queued_task() #Get no. of tasks on list 

      if active_task < task_count: #Check if there are finished tasks
        task_count = active_task
        print("*"*30)
        print("Number of current pending tasks in queue: {: }.".format(task_count))
        print("Remaining tasks before starting new batch: {: }.format(task_count-target_count")

      else: #check whether new imagery to be pooled is alr in google drive or in queue
        if (os.path.exists(imagery_filepath)==False): #prevents duplication of tasks
          if(imagery_file not in queued_filenames): 
            print("-----------------------")
            print("Starting new task...")
            print("downloading "+ imagery_file)

            #Set var to store centroid coordinates obtained from centroid CSV
            c_lon = df['lon'][i]
            c_lat = df['lat'][i]
           
            #Employ centroid coordinates to define a geospatial circle using GEE point geometry
            #Buffer of 1920m - 1/2 grid size measured from centroid to grid boundary
            geometry = ee.Geometry.Point([c_lon, c_lat]).buffer(1920)

            #Redefine geom var with c of circle as its value
            geometry = geometry.getInfo()['coordinates'][0]
            
            #Define export parameters
            if(day_sat == "ST"):
              scale = 10 
            elif (day_sat == "LS"): 
              scale =15 

            task_config = {
                'scale': scale, #satellite resolution 
                'region': geometry, #area coverage to download
                'driveFolder': drive_folder #folder path
            }
            
            #Describe image batch export object & name as task
            task = ee.batch.Export.image(sat_imagery, imagery_file, task_config)
            task.start() #Pass the task to GEE
            task_count += 1

            if task_count %1000 == 0: 
              task_count = get_queued_task() #Execute get_queued_tasks only after every 1000 tasks

            print('Number of active tasks: {: }.'.format(task_count))
        
        else: 
          print("On queue: " + imagery_file + ".tif")

      else: 
        print("Downloaded: " + imagery_file +".tif")



As illustrated in Step 6 of the section on Generating Centroids for Satellite imagery, buffer size is computed
as follows:

256 pixel x 15 meters/pixel = 3840 meter grid size

3840 / 2 = 1920 meter buffer size

where: 15 meters/pixel is the Landsat resolution

In [ ]:
download_satellite_imagery(satellite_imagery)